## 1. 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

"""
[train.csv]                                         [test.csv]
ID : 샘플별 고유 ID                                 ID : 샘플별 고유 ID
gender : 성별                                       gender : 성별
age : 연령                                          age : 연령
height : 키(cm)                                     height : 키(cm)
weight : 몸무게(kg)                                 weight : 몸무게(kg)
cholesterol : 콜레스테롤 수치                       cholesterol : 콜레스테롤 수치
systolic_blood_pressure : 수축기 혈압               systolic_blood_pressure : 수축기 혈압
diastolic_blood_pressure : 이완기 혈압              diastolic_blood_pressure : 이완기 혈압
glucose : 혈당 수치(mg/dL)                          glucose : 혈당 수치(mg/dL)
bone_density : 골밀도(g/cm²)                        bone_density : 골밀도(g/cm²)
activity : 생활시 운동 강도                         activity : 생활시 운동 강도
smoke_status : 흡연 상태                            smoke_status : 흡연 상태
medical_history : 만성질환                          medical_history : 만성질환
family_medical_history : 가족력                     family_medical_history : 가족력
sleep_pattern : 수면패턴                            sleep_pattern : 수면패턴
edu_level : 학력                                    edu_level : 학력
mean_working : 1주일당 평균 근로 시간               mean_working : 1주일당 평균 근로 시간
stress_score : (TARGET) 스트레스 점수                    
"""

display(train.shape)

(3000, 18)

## 2. 데이터 전처리

In [3]:
# 카테고리형 변수의 결측값을 최빈값으로 대체
# 최빈값 -> 가장 많이 관측된 값
categorical_na_cols = train.select_dtypes(include=['object', 'category']).columns.drop('ID')

for col in categorical_na_cols:
    # # 학습 데이터에서 최빈값 계산
    # most_frequent = train[col].mode()[0]
    # print(f"{col}: {most_frequent}")
    # # 학습 데이터와 테스트 데이터 모두 해당 칼럼의 최빈값으로 대체
    # train[col] = train[col].fillna(most_frequent)
    # test[col] = test[col].fillna(most_frequent)

    train[col] = train[col].fillna('None')
    test[col] = test[col].fillna('None')

    print(f"{col}: {train[col].unique()}")

gender: ['F' 'M']
activity: ['moderate' 'light' 'intense']
smoke_status: ['ex-smoker' 'non-smoker' 'current-smoker']
medical_history: ['high blood pressure' 'None' 'diabetes' 'heart disease']
family_medical_history: ['diabetes' 'None' 'high blood pressure' 'heart disease']
sleep_pattern: ['sleep difficulty' 'normal' 'oversleeping']
edu_level: ['bachelors degree' 'graduate degree' 'high school diploma' 'None']


In [6]:
# mean_working이 NaN이거나 0이면 0, 아니면 1로 is_working 컬럼 추가
train['is_working'] = ((train['mean_working'].fillna(0)) != 0).astype(int)
test['is_working'] = ((test['mean_working'].fillna(0)) != 0).astype(int)

In [7]:
# 1. 컬럼 목록에서 is_working이 있는지 확인
print('is_working' in train.columns)  # True면 정상 추가

# 2. train 데이터프레임의 상위 5개 행에서 is_working 컬럼 확인
print(train[['mean_working', 'is_working']].head())

# 3. is_working 컬럼의 값 분포(0/1 개수) 확인
print(train['is_working'].value_counts())

True
   mean_working  is_working
0           NaN           0
1           NaN           0
2           9.0           1
3           NaN           0
4           NaN           0
is_working
1    1968
0    1032
Name: count, dtype: int64


In [8]:
# mean_working에 대해 중앙값 대체
# median_value = train['mean_working'].median()

# mean_working 결측치 -> 0
# train['mean_working'] = train['mean_working'].fillna(0)
# test['mean_working'] = test['mean_working'].fillna(0)

# mean_working 결측치를 평균값으로 대체
mean_value = train['mean_working'].mean()
train['mean_working'] = train['mean_working'].fillna(mean_value)
test['mean_working'] = test['mean_working'].fillna(mean_value)

display(train.columns)

Index(['ID', 'gender', 'age', 'height', 'weight', 'cholesterol',
       'systolic_blood_pressure', 'diastolic_blood_pressure', 'glucose',
       'bone_density', 'activity', 'smoke_status', 'medical_history',
       'family_medical_history', 'sleep_pattern', 'edu_level', 'mean_working',
       'stress_score', 'is_working'],
      dtype='object')

In [9]:
# One-Hot Encoding 적용 열 - gender, smoke_status, medical_history, family_medical_history
# 원핫 인코딩 => 카테고리끼리 순서 없을때 사용
# smoke_status는 좀 애매하긴 한듯
def one_hot_encoding(df):
    one_hot_encoding_cols = ['gender', 'smoke_status', 'medical_history', 'family_medical_history']
    encoding_df = pd.get_dummies(df, columns=one_hot_encoding_cols)
    return encoding_df

train = one_hot_encoding(train)
test = one_hot_encoding(test)

display(train.columns)
display(train.shape)

Index(['ID', 'age', 'height', 'weight', 'cholesterol',
       'systolic_blood_pressure', 'diastolic_blood_pressure', 'glucose',
       'bone_density', 'activity', 'sleep_pattern', 'edu_level',
       'mean_working', 'stress_score', 'is_working', 'gender_F', 'gender_M',
       'smoke_status_current-smoker', 'smoke_status_ex-smoker',
       'smoke_status_non-smoker', 'medical_history_None',
       'medical_history_diabetes', 'medical_history_heart disease',
       'medical_history_high blood pressure', 'family_medical_history_None',
       'family_medical_history_diabetes',
       'family_medical_history_heart disease',
       'family_medical_history_high blood pressure'],
      dtype='object')

(3000, 28)

In [10]:
# Label Encoding 적용 열 - activity, sleep_pattern, edu_level
# 라벨 인코딩 => 카테고리끼리 순서 있을때 사용
def label_encoding(df):
    label_encoding_cols = ['activity', 'sleep_pattern', 'edu_level']
    for feature in label_encoding_cols:
        le = LabelEncoder()
        le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

train = label_encoding(train)
test = label_encoding(test)

In [11]:
# 모든 bool 컬럼을 int로 변환
for col in train.columns:
    if train[col].dtype == 'bool':
        train[col] = train[col].astype(int)
for col in test.columns:
    if test[col].dtype == 'bool':
        test[col] = test[col].astype(int)

In [12]:
train

,ID,age,height,weight,cholesterol,systolic_blood_pressure,diastolic_blood_pressure,glucose,bone_density,activity,...,smoke_status_ex-smoker,smoke_status_non-smoker,medical_history_None,medical_history_diabetes,medical_history_heart disease,medical_history_high blood pressure,family_medical_history_None,family_medical_history_diabetes,family_medical_history_heart disease,family_medical_history_high blood pressure
0,TRAIN_0000,72,161.49,58.47,279.84,165,100,143.35,0.87,2,...,1,0,0,0,0,1,0,1,0,0
1,TRAIN_0001,88,179.87,77.60,257.37,178,111,146.94,0.07,2,...,1,0,1,0,0,0,0,1,0,0
2,TRAIN_0002,47,182.47,89.93,226.66,134,95,142.61,1.18,1,...,1,0,1,0,0,0,1,0,0,0
3,TRAIN_0003,69,185.78,68.63,206.74,158,92,137.26,0.48,0,...,1,0,0,0,0,1,1,0,0,0
4,TRAIN_0004,81,164.63,71.53,255.92,171,116,129.37,0.34,2,...,1,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,TRAIN_2995,87,174.43,77.14,233.31,176,111,134.71,0.20,1,...,0,1,1,0,0,0,1,0,0,0
2996,TRAIN_2996,25,171.44,51.67,196.89,143,86,111.91,1.75,2,...,0,0,1,0,0,0,1,0,0,0
2997,TRAIN_2997,54,162.13,55.54,210.52,164,91,136.47,0.80,1,...,0,0,1,0,0,0,1,0,0,0
2998,TRAIN_2998,70,181.32,82.20,234.10,182,91,144.89,0.58,2,...,0,1,1,0,0,0,1,0,0,0


## 3. 피처 엔지니어링

In [13]:
# BMI => 몸무게[kg] / 키[cm]^2
# 혈압차(맥압) => 수축성 혈압 - 이완성 혈압
def add_features(df):
    df['BMI'] = df['weight'] / ((df['height']/100)**2)
    df['bp_diff'] = df['systolic_blood_pressure'] - df['diastolic_blood_pressure']
    return df

train = add_features(train)
test = add_features(test)

In [14]:
display(train['mean_working'].describe())

# working_group 피처 만들기
bins = [3, 8, 9, 16]  # 적절한 경계값 설정
labels = [0, 1, 2]     # 저, 중하, 중상, 고 근무

train['working_group'] = pd.cut(train['mean_working'], bins=bins, labels=labels, include_lowest=True)
test['working_group'] = pd.cut(test['mean_working'], bins=bins, labels=labels, include_lowest=True)

overwork_threshold = train['mean_working'].quantile(0.95)  # 상위 5% 기준값
train['is_overwork'] = (train['mean_working'] >= overwork_threshold).astype(int)
test['is_overwork'] = (test['mean_working'] >= overwork_threshold).astype(int)

display(train['working_group'].value_counts())
display(train['is_overwork'].value_counts())

count    3000.000000
mean        8.716972
std         1.319229
min         4.000000
25%         8.000000
50%         8.716972
75%         9.000000
max        16.000000
Name: mean_working, dtype: float64

working_group
1    1569
0     888
2     543
Name: count, dtype: int64

is_overwork
0    2803
1     197
Name: count, dtype: int64

In [15]:
# mean_working & sleep_pattern 피처 엔지니어링

train['work_sleep_imbalance'] = train['mean_working'] / (train['sleep_pattern'] + 1)
test['work_sleep_imbalance'] = test['mean_working'] / (test['sleep_pattern'] + 1)

display(train[['work_sleep_imbalance', 'stress_score']])

for col in train.columns:
    if col not in ['ID', 'stress_score']:
        corr_val = train['stress_score'].corr(train[col])
        print(f"{col} ↔ stress_score 상관계수: {corr_val:.4f}")

,work_sleep_imbalance,stress_score
0,2.905657,0.63
1,8.716972,0.83
2,9.000000,0.70
3,4.358486,0.17
4,2.905657,0.36
...,...,...
2995,4.358486,0.02
2996,9.000000,0.16
2997,9.000000,0.16
2998,4.358486,0.18


age ↔ stress_score 상관계수: 0.0187
height ↔ stress_score 상관계수: -0.0057
weight ↔ stress_score 상관계수: 0.0113
cholesterol ↔ stress_score 상관계수: 0.0213
systolic_blood_pressure ↔ stress_score 상관계수: 0.0156
diastolic_blood_pressure ↔ stress_score 상관계수: 0.0254
glucose ↔ stress_score 상관계수: -0.0061
bone_density ↔ stress_score 상관계수: -0.0226
activity ↔ stress_score 상관계수: 0.0070
sleep_pattern ↔ stress_score 상관계수: 0.0002
edu_level ↔ stress_score 상관계수: -0.0271
mean_working ↔ stress_score 상관계수: 0.1507
is_working ↔ stress_score 상관계수: -0.0228
gender_F ↔ stress_score 상관계수: 0.0136
gender_M ↔ stress_score 상관계수: -0.0136
smoke_status_current-smoker ↔ stress_score 상관계수: 0.0261
smoke_status_ex-smoker ↔ stress_score 상관계수: -0.0320
smoke_status_non-smoker ↔ stress_score 상관계수: 0.0087
medical_history_None ↔ stress_score 상관계수: -0.0504
medical_history_diabetes ↔ stress_score 상관계수: 0.0245
medical_history_heart disease ↔ stress_score 상관계수: 0.0000
medical_history_high blood pressure ↔ stress_score 상관계수: 0.0373
family_medical

## 4. 모델링

In [16]:
from sklearn.model_selection import train_test_split

X = train.drop(['ID', 'stress_score'], axis=1)
y = train['stress_score']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# x_train = train.drop(['ID', 'stress_score'], axis = 1)
# x_val = train['stress_score']
# y_train = train['stress_score']

test = test.drop('ID', axis = 1)

In [18]:
# lgbm = LGBMRegressor(random_state = 42)
# lgbm.fit(x_train, y_train)

# pred = lgbm.predict(test)

In [19]:
# category 타입 컬럼을 int로 변환
for col in X_train.columns:
    if str(X_train[col].dtype) == 'category':
        X_train[col] = X_train[col].astype(int)
        X_val[col] = X_val[col].astype(int)

# train, test의 category 타입 컬럼을 int로 변환
for col in train.columns:
    if str(train[col].dtype) == 'category':
        train[col] = train[col].astype(int)
for col in test.columns:
    if str(test[col].dtype) == 'category':
        test[col] = test[col].astype(int)

In [20]:
import optuna
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

def objective(trial):
    # 하이퍼파라미터 샘플링
    lgbm_params = {
        'n_estimators': trial.suggest_int('lgbm_n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('lgbm_learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('lgbm_num_leaves', 20, 100),
        'random_state': 42
    }
    xgb_params = {
        'n_estimators': trial.suggest_int('xgb_n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('xgb_learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('xgb_max_depth', 3, 10),
        'random_state': 42
    }
    cat_params = {
        'iterations': trial.suggest_int('cat_iterations', 100, 1000),
        'learning_rate': trial.suggest_float('cat_learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('cat_depth', 3, 10),
        'random_seed': 42,
        'verbose': 0
    }

    # 모델 학습
    lgbm = LGBMRegressor(**lgbm_params)
    xgb = XGBRegressor(**xgb_params)
    cat = CatBoostRegressor(**cat_params)

    lgbm.fit(X_train, y_train)
    xgb.fit(X_train, y_train)
    cat.fit(X_train, y_train)

    # 예측
    pred_lgbm = lgbm.predict(X_val)
    pred_xgb = xgb.predict(X_val)
    pred_cat = cat.predict(X_val)

    # 앙상블 가중치 샘플링 및 정규화
    w_lgbm = trial.suggest_float('w_lgbm', 0, 1)
    w_xgb = trial.suggest_float('w_xgb', 0, 1)
    w_cat = trial.suggest_float('w_cat', 0, 1)
    total = w_lgbm + w_xgb + w_cat
    w_lgbm /= total
    w_xgb /= total
    w_cat /= total

    # 앙상블 예측
    pred_ensemble = w_lgbm * pred_lgbm + w_xgb * pred_xgb + w_cat * pred_cat

    # MAE 계산
    mae = mean_absolute_error(y_val, pred_ensemble)
    return mae

# Optuna Study 생성 및 최적화
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print('Best trial:')
print(study.best_trial)
print('Best params:', study.best_trial.params)

[I 2025-07-23 22:46:29,301] A new study created in memory with name: no-name-5992b53e-d579-4b58-acbd-70cd9bf9a66a


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000209 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1834
[LightGBM] [Info] Number of data points in the train set: 2400, number of used features: 31
[LightGBM] [Info] Start training from score 0.481912


[I 2025-07-23 22:46:33,359] Trial 0 finished with value: 0.17805412300276183 and parameters: {'lgbm_n_estimators': 200, 'lgbm_learning_rate': 0.2380929064513637, 'lgbm_num_leaves': 24, 'xgb_n_estimators': 709, 'xgb_learning_rate': 0.2176778754078496, 'xgb_max_depth': 3, 'cat_iterations': 253, 'cat_learning_rate': 0.08700465236480157, 'cat_depth': 9, 'w_lgbm': 0.8484450625046581, 'w_xgb': 0.017097546201997882, 'w_cat': 0.8680394893324958}. Best is trial 0 with value: 0.17805412300276183.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000248 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1834
[LightGBM] [Info] Number of data points in the train set: 2400, number of used features: 31
[LightGBM] [Info] Start training from score 0.481912


[I 2025-07-23 22:46:35,407] Trial 1 finished with value: 0.19401169550204178 and parameters: {'lgbm_n_estimators': 687, 'lgbm_learning_rate': 0.02258560628412292, 'lgbm_num_leaves': 85, 'xgb_n_estimators': 697, 'xgb_learning_rate': 0.07459352603785672, 'xgb_max_depth': 5, 'cat_iterations': 173, 'cat_learning_rate': 0.13035545360245196, 'cat_depth': 5, 'w_lgbm': 0.517739420120429, 'w_xgb': 0.10346682329568502, 'w_cat': 0.8133431564654362}. Best is trial 0 with value: 0.17805412300276183.


...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000229 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1834
[LightGBM] [Info] Number of data points in the train set: 2400, number of used features: 31
[LightGBM] [Info] Start training from score 0.481912


[I 2025-07-23 22:50:48,083] Trial 48 finished with value: 0.16130273001505413 and parameters: {'lgbm_n_estimators': 164, 'lgbm_learning_rate': 0.19895196162082854, 'lgbm_num_leaves': 45, 'xgb_n_estimators': 696, 'xgb_learning_rate': 0.14333552365175503, 'xgb_max_depth': 10, 'cat_iterations': 884, 'cat_learning_rate': 0.14468764366716041, 'cat_depth': 8, 'w_lgbm': 0.7505228671773148, 'w_xgb': 0.5192950338646558, 'w_cat': 0.8627459136063318}. Best is trial 48 with value: 0.16130273001505413.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000170 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1834
[LightGBM] [Info] Number of data points in the train set: 2400, number of used features: 31
[LightGBM] [Info] Start training from score 0.481912


[I 2025-07-23 22:50:52,426] Trial 49 finished with value: 0.16289459782513732 and parameters: {'lgbm_n_estimators': 179, 'lgbm_learning_rate': 0.2035460051951775, 'lgbm_num_leaves': 44, 'xgb_n_estimators': 559, 'xgb_learning_rate': 0.14548592763763513, 'xgb_max_depth': 9, 'cat_iterations': 909, 'cat_learning_rate': 0.15311852438967155, 'cat_depth': 8, 'w_lgbm': 0.44693216406077163, 'w_xgb': 0.5263054258640745, 'w_cat': 0.9159436171739392}. Best is trial 48 with value: 0.16130273001505413.


Best trial:
FrozenTrial(number=48, state=1, values=[0.16130273001505413], datetime_start=datetime.datetime(2025, 7, 23, 22, 50, 43, 963265), datetime_complete=datetime.datetime(2025, 7, 23, 22, 50, 48, 83603), params={'lgbm_n_estimators': 164, 'lgbm_learning_rate': 0.19895196162082854, 'lgbm_num_leaves': 45, 'xgb_n_estimators': 696, 'xgb_learning_rate': 0.14333552365175503, 'xgb_max_depth': 10, 'cat_iterations': 884, 'cat_learning_rate': 0.14468764366716041, 'cat_depth': 8, 'w_lgbm': 0.7505228671773148, 'w_xgb': 0.5192950338646558, 'w_cat': 0.8627459136063318}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lgbm_n_estimators': IntDistribution(high=1000, log=False, low=100, step=1), 'lgbm_learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'lgbm_num_leaves': IntDistribution(high=100, log=False, low=20, step=1), 'xgb_n_estimators': IntDistribution(high=1000, log=False, low=100, step=1), 'xgb_learning_rate': FloatDistribution(high=0.3, log

In [ ]:
# 1. 최적 파라미터 추출
best_params = study.best_trial.params

# 2. 모델 생성 (최적 파라미터 적용)
lgbm = LGBMRegressor(
    n_estimators=best_params['lgbm_n_estimators'],
    learning_rate=best_params['lgbm_learning_rate'],
    num_leaves=best_params['lgbm_num_leaves'],
    random_state=42
)
xgb = XGBRegressor(
    n_estimators=best_params['xgb_n_estimators'],
    learning_rate=best_params['xgb_learning_rate'],
    max_depth=best_params['xgb_max_depth'],
    random_state=42
)
cat = CatBoostRegressor(
    iterations=best_params['cat_iterations'],
    learning_rate=best_params['cat_learning_rate'],
    depth=best_params['cat_depth'],
    random_seed=42,
    verbose=0
)

# 3. 전체 train 데이터로 재학습
X_full = train.drop(['ID', 'stress_score'], axis=1, errors='ignore')
y_full = train['stress_score']

lgbm.fit(X_full, y_full)
xgb.fit(X_full, y_full)
cat.fit(X_full, y_full)

# 4. test 데이터 예측
pred_lgbm = lgbm.predict(test)
pred_xgb = xgb.predict(test)
pred_cat = cat.predict(test)


In [ ]:
# 5. 앙상블 가중치 정규화
w_lgbm = best_params['w_lgbm']
w_xgb = best_params['w_xgb']
w_cat = best_params['w_cat']
total = w_lgbm + w_xgb + w_cat
w_lgbm /= total
w_xgb /= total
w_cat /= total

# 6. 앙상블 예측
pred_ensemble = w_lgbm * pred_lgbm + w_xgb * pred_xgb + w_cat * pred_cat

In [ ]:
# 7. 제출 파일 생성
submission = pd.read_csv('./sample_submission.csv')
submission['stress_score'] = pred_ensemble
submission.to_csv('submit.csv', index=False)

In [ ]:
# importances = lgbm.feature_importances_
# feature_names = x_train.columns

# importance_df = pd.DataFrame({
#     'Feature': feature_names,
#     'Importance': importances
# }).sort_values(by='Importance', ascending=False)

# plt.figure(figsize=(10, 6))
# sns.barplot(data=importance_df.head(20), x='Importance', y='Feature')
# plt.title('Top 20 Feature Importances (LGBM)')
# plt.show()

In [ ]:
submission = pd.read_csv('./sample_submission.csv')

In [ ]:
submission['stress_score'] = pred
submission.head()

In [ ]:
submission.to_csv('submit.csv', index=False)

In [ ]:
# train에서 나눠서 데이콘에서 측정하는 MAE와 다를수있음
# 성능이 올라갔는지 그대로인지 내려갔는지 경향만 파악할 수 있음
# 확인용으로만 쓸것

import datetime

# MAE 로그파일 세팅
def loging(MAE):
    with open('./mae_log', mode='a', encoding='utf-8') as f:
        current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        f.write(f"시간: {current_time} / MAE: {MAE}\n")

# MAE 체크용
train_set, val_set = train_test_split(train, test_size=0.2, random_state=42)

X_train = train_set.drop(['ID', 'stress_score'], axis=1)
y_train = train_set['stress_score']

X_val = val_set.drop(['ID', 'stress_score'], axis=1) 
y_val = val_set['stress_score']

lgbm = LGBMRegressor(random_state=42)
lgbm.fit(X_train, y_train)

val_pred = lgbm.predict(X_val)

MAE = mean_absolute_error(y_val, val_pred)

print(f"모델의 검증 MAE 점수: {MAE}")
loging(MAE)